In [1]:
import os

In [2]:
# use GPU 1
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
IMAGES_DIR = '/d2/caches/kaggle-planet/processed-images-64x64'
IMAGE_SHAPE = (64, 64, 4)
TEST_TIFFS_DIR = '/datasets/kaggle/planet/test-tif-v2'
MODEL = '/d2/caches/kaggle-planet/models/A/e09-l=0.22781-vl=0.28093.h5'
THRESHOLD = 0.5

In [5]:
%run 'Lib.ipynb'

MODEL_CHECKPOINT_DIR not defined 


In [6]:
import keras

In [7]:
X_files_pred = [x.split('.')[0] for x in list_files(TEST_TIFFS_DIR)]
print(len(X_files_pred))

61191


In [8]:
model = keras.models.load_model(MODEL)

In [9]:
subm = []

import re

classes_misc = np.load('out/classes_misc.npy').tolist()
classes_weather = np.load('out/classes_weather.npy').tolist()

i = 0

for f in tqdm(X_files_pred):
    assert (os.path.isfile(IMAGES_DIR + '/' + f))

    b = generate_batch(
        n_samples=1,
        batch_index=i,
        X_files=X_files_pred,
        Y_tags_misc=None,
        Y_tags_weather=None,
        images_dir=IMAGES_DIR,
        image_shape=IMAGE_SHAPE)

    p = model.predict_on_batch(b)

    tags = []

    tags_misc = p[0][0]
    tags_weather = p[1][0]

    # misc
    for j, v in enumerate(tags_misc):
        if v > THRESHOLD:  # cut-off
            tags.append(classes_misc[j])

    # weather
    tag_weather = classes_weather[np.argmax(tags_weather)]
    if 'unknown' != tag_weather: tags.append(tag_weather)

    subm.append({'image_name': f, 'tags': ' '.join(tags)})

    i += 1

100%|██████████| 61191/61191 [01:51<00:00, 549.15it/s]


In [10]:
subm_df = pd.DataFrame(subm)

In [11]:
subm_df.to_csv('out/submission-c.csv', index=False)